# Importing the required libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span",attrs={"id":'productTitle'}).text.strip()

    except AttributeError:
        title= ""

    return title

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span",attrs={"class":"a-price-whole"}).text

    except AttributeError:
         price = ""
    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon-alt'}).string.strip()
    
    except AttributeError:
        rating = ""	

    return rating


# Function to extract SellerName
def get_sellername(soup):
    try:
        sellername = soup.find("div",attrs={"id":"merchant-info" }).text.strip()
        

    except AttributeError:
        sellername = "Out Of Stock"	

    return sellername

In [ ]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?rh=n%3A6612025031&fs=true&ref=lp_6612025031_sar"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[],"SellerName":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['SellerName'].append(get_sellername(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)
